In [8]:
import asyncio
import aiohttp
import pandas as pd

BASE_URL = "https://api.investing.com/api/financialdata/assets/equitiesByCountry/default"

PAGE_SIZE = 100
CONCURRENT_PAGES = 6

PARAMS = {
    # 👇 request all required fields
    "fields-list": "url,volume,fundamentalMarketCap,name,symbol",
    "country-id": 14,
    "filter-domain": "",
    "page-size": PAGE_SIZE,
    "limit": 0,
    "include-additional-indices": "false",
    "include-major-indices": "false",
    "include-other-indices": "false",
    "include-primary-sectors": "false",
    "include-market-overview": "false"
}

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept": "application/json, text/plain, */*",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.investing.com/",
    "Origin": "https://www.investing.com"
}

async def fetch_page(session, page):
    params = PARAMS.copy()
    params["page"] = page

    async with session.get(BASE_URL, params=params, headers=HEADERS) as resp:
        if resp.status != 200:
            return []
        js = await resp.json()
        return js.get("data", [])

async def main():
    all_data = []
    page = 0

    async with aiohttp.ClientSession(
        timeout=aiohttp.ClientTimeout(total=40)
    ) as session:

        while True:
            tasks = [
                fetch_page(session, page + i)
                for i in range(CONCURRENT_PAGES)
            ]

            results = await asyncio.gather(*tasks)

            stop = False
            for data in results:
                if not data:
                    stop = True
                else:
                    all_data.extend(data)
                    if len(data) < PAGE_SIZE:
                        stop = True

            if stop:
                break

            page += CONCURRENT_PAGES

    print("Total records fetched:", len(all_data))

    # 🔽 BUILD DATAFRAME
    df = pd.DataFrame(all_data)

    # normalize column names
    df.columns = [c.lower() for c in df.columns]

    required_columns = [
        "url",
        "volume",
        "fundamentalmarketcap",
        "name",
        "symbol"
    ]

    missing = [c for c in required_columns if c not in df.columns]
    if missing:
        raise RuntimeError(f"❌ Missing columns: {missing}")

    df = df[required_columns]

    # optional: nicer column names
    df.rename(
        columns={
            "fundamentalmarketcap": "market_cap"
        },
        inplace=True
    )

    df.to_csv("investing_equities_data.csv", index=False)

    print("✅ Saved: investing_equities_data.csv")
    return df

# ▶ Run in Jupyter
await main()

Total records fetched: 7544
✅ Saved: investing_equities_data.csv


,url,volume,market_cap,name,symbol
0,/equities/aditya-birla,5757878,8.839300e+11,Aditya Birla Capital,ADTB
1,/equities/hubtown,761265,2.922000e+10,Hubtown,HUBT
2,/equities/3i-infotech,256725,3.100000e+09,3i Infotech,TIIN
3,/equities/3m-india,4490,3.869000e+11,3M India,TMIN
4,/equities/abb-limited,634906,1.180000e+12,ABB India,ABB
...,...,...,...,...,...
7539,/equities/shankara-buildpro?cid=1238282,1492,1.731000e+10,Shankara Buildpro,SNKR
7540,/equities/amagi-media-labs?cid=1238283,333607,7.860000e+10,Amagi Media Labs,AMAI
7541,/equities/sacheerome,5400,8.330000e+09,Sacheerome,SACR
7542,/equities/shadowfax-technologies-ltd,7085928,NaN,Shadowfax Technologies,SADO


In [12]:
df = pd.read_csv("investing_equities_data.csv")

In [18]:
df["news_url"] = df["url"].str.replace(
    r'^([^?]+)(\?.*)?$',
    r'\1-news\2',
    regex=True
)

df = df.sort_values(by="market_cap", ascending=False).reset_index(drop=True)

In [22]:
df.to_csv("investing_equities_news_data.csv", index = False)

In [29]:
import pandas as pd

# read data (tab-separated as per your sample)
df = pd.read_csv("investing_equities_news_data.csv")
df.columns = ["url", "volume", "market_cap", "company", "symbol", "news_url"]

# flag cid rows
df["has_cid"] = df["url"].str.contains(r"\?cid=")

# sort so that non-cid rows come first within each symbol
df = df.sort_values(by=["symbol", "has_cid"])

# drop duplicates on symbol, keep the non-cid one
df_dedup = df.drop_duplicates(subset="symbol", keep="first")

# cleanup
df_dedup = df_dedup.drop(columns="has_cid")
df_dedup = df_dedup.sort_values(by="market_cap", ascending=False).reset_index(drop=True)
print(df_dedup)


                                               url    volume    market_cap  \
0                    /equities/reliance-industries  11239835  1.891000e+13   
1                          /equities/hdfc-bank-ltd  29691233  1.430000e+13   
2              /equities/bharti-airtel?cid=1191427    204688  1.201000e+13   
3                          /equities/bharti-airtel   7683747  1.201000e+13   
4              /equities/tata-consultancy-services   3022068  1.130000e+13   
...                                            ...       ...           ...   
5423       /equities/shri-hare-krishna-sponge-iron      4000           NaN   
5424         /equities/shri-gang-industries-allied      1027           NaN   
5425  /equities/sustainable-energy-infra-trust-unt     25000           NaN   
5426      /equities/unified-data-tech-solutions-bo     16800           NaN   
5427                    /equities/varun-mercantile         0           NaN   

                                 company  symbol  \
0          

In [30]:
df_dedup.head()

,url,volume,market_cap,company,symbol,news_url
0,/equities/reliance-industries,11239835,1.891000e+13,Reliance Industries,RELI,/equities/reliance-industries-news
1,/equities/hdfc-bank-ltd,29691233,1.430000e+13,HDFC Bank,HDBK,/equities/hdfc-bank-ltd-news
2,/equities/bharti-airtel?cid=1191427,204688,1.201000e+13,Bharti Airtel,BRTIpp,/equities/bharti-airtel-news?cid=1191427
3,/equities/bharti-airtel,7683747,1.201000e+13,Bharti Airtel,BRTI,/equities/bharti-airtel-news
4,/equities/tata-consultancy-services,3022068,1.130000e+13,Tata Consultancy,TCS,/equities/tata-consultancy-services-news


In [ ]:
df_dedup.to_csv()